# 筆記本 6：事件地點匹配與輸出

本節將從地名資料庫中查詢對應的地理座標，並匹配到對應的事件資料中。這些事件資料會包含地點、事件類型、時間以及其他補充資訊。這些數據將被組合並保存到新的 CSV 文件中，以便後續分析。

## 目標
- 匯入地名資料庫。
- 從事件資料中查找對應的地點資訊。
- 將事件資料與地點的 GPS 座標匹配並輸出。


In [ ]:
# 安裝相關套件
!pip install pandas

### 步驟 1：載入地名資料庫
首先，我們需要從地名資料庫中加載含有地點及其經緯度的數據。

In [ ]:
import csv
import pandas as pd

def load_place_data(database_file):
    """
    載入地點資料庫，將每個地名和對應的建議地名及經緯度資訊儲存到字典中。
    """
    place_data = {}
    with open(database_file, mode='r', encoding='utf-8') as db_file:
        csv_reader = csv.DictReader(db_file, delimiter='|')
        for row in csv_reader:
            # 將整行資料儲存，以便地名和建議地名的使用
            place_data[row['地名']] = {
                'original_name': row['地名'],
                'suggested_name': row['建議地名'],
                'latitude': row['緯度'],
                'longitude': row['經度']
            }
    return place_data

# 測試載入地名資料庫
place_data = load_place_data('place_db.csv')  # 假設地名資料庫文件的名稱
print(list(place_data.items())[:5])  # 印出前 5 筆資料以檢查

### 步驟 2：處理事件資料並匹配地點資訊
接下來，我們將會從事件資料中讀取事件地點，並根據其名稱從地點資料庫中查找對應的經緯度。

In [ ]:
def process_event_data(input_file, place_data):
    """
    處理輸入檔案，依據地點名稱匹配資料庫中的地名並組合事件資訊。
    """
    processed_events = []
    with open(input_file, mode='r', encoding='utf-8') as in_file:
        csv_reader = csv.DictReader(in_file)
        for row in csv_reader:
            event_type = row['事件類型']
            event_date = row['日期']
            location = row['地點']
            additional_info = row['額外說明']

            # 確認地點是否在地點資料庫中
            if location in place_data:
                place_info = place_data[location]
                combined_description = f"{event_date}_{event_type}_{location}_{additional_info}"
                processed_events.append({
                    'name': place_info['original_name'],  # 使用地名（原始名稱）
                    'longitude': place_info['longitude'],
                    'latitude': place_info['latitude'],
                    'description': combined_description
                })
    return processed_events

# 測試處理事件資料
processed_events = process_event_data('event_log.txt', place_data)  # 假設事件資料文件名稱為 event_data.csv
print(processed_events[:5])  # 顯示前 5 筆處理結果

### 步驟 3：將處理結果保存為 CSV 檔案
我們將把匹配到的事件資料和地點寫入新的 CSV 檔案。

In [ ]:
def save_to_csv(output_file, processed_events):
    """
    將處理後的事件資料儲存至輸出檔案。
    """
    with open(output_file, mode='w', newline='', encoding='utf-8') as out_file:
        fieldnames = ['name', 'longitude', 'latitude', 'description']
        csv_writer = csv.DictWriter(out_file, fieldnames=fieldnames)
        csv_writer.writeheader()
        for event in processed_events:
            csv_writer.writerow(event)

# 將結果寫入輸出檔案
save_to_csv('output_matched_events.csv', processed_events)  # 假設輸出文件名稱為 output_matched_events.csv
print('匹配結果已保存。')

### 結論
在這個案例中，我們使用了一個地名資料庫來匹配事件的地點，並提取了相關的座標資訊。最終，我們將這些匹配結果輸出到一個 CSV 檔案，這可以應用於地點相關的分析中，例如地理可視化、事件分析等。

### 可以進行的後續操作
- 可以將匹配結果進行進一步的地圖可視化。
- 可結合時間數據，進行地點與事件頻率的趨勢分析等。